# TensorBoard 测试

## 删除 log 目录

In [14]:
# 删除 log 目录
import os
import shutil

dirpath = '.\logs'
if os.path.exists(dirpath) and os.path.isdir(dirpath):
    shutil.rmtree(dirpath)

## 模型

In [15]:
import tensorflow as tf

mnist = tf.keras.datasets.mnist

# 载入 MNIST 手写阿拉伯数字资料
(x_train, y_train),(x_test, y_test) = mnist.load_data()

# 特征缩放，使用常态化(Normalization)，公式 = (x - min) / (max - min)
x_train_norm, x_test_norm = x_train / 255.0, x_test / 255.0

# 建立模型
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])


## 设定优化器(optimizer)、损失函数(loss)、效能衡量指标(metrics)

In [16]:
# 设定优化器(optimizer)、损失函数(loss)、效能衡量指标(metrics)的类别
loss_function = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Define 训练及测试的效能衡量指标(Metrics)
train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')
test_loss = tf.keras.metrics.Mean('test_loss', dtype=tf.float32)
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy')

## 定义模型训练的函数

In [17]:
def train_step(model, optimizer, x_train, y_train):
    # 自动微分
    with tf.GradientTape() as tape:
        predictions = model(x_train, training=True)
        loss = loss_function(y_train, predictions)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    # 计算训练的效能衡量指标
    train_loss(loss)
    train_accuracy(y_train, predictions)

def test_step(model, x_test, y_test):
    # 预测
    predictions = model(x_test)
    # 计算损失
    loss = loss_function(y_test, predictions)

    # 计算测试的效能衡量指标
    test_loss(loss)
    test_accuracy(y_test, predictions)

## 设定 log 目录，开启 log 档案

In [18]:
import datetime

current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# 指定训练的 log 档名
train_log_dir = '.\\logs\\gradient_tape\\' + current_time + '\\train'
# 指定测试的 log 档名
test_log_dir = '.\\logs\\gradient_tape\\' + current_time + '\\test'

# 开启 log 档案
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

## 将训练/测试资料转成 Tensorflow Dataset

In [19]:
# 将训练/测试资料转成 Tensorflow Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((x_train_norm, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test_norm, y_test))

# 每次从 60000 笔训练资料随机抽出 64 笔
# shuffle：洗牌，batch：每批 64 笔
train_dataset = train_dataset.shuffle(60000).batch(64)
# 每次从 10000 笔测试资料随机抽出 64 笔
test_dataset = test_dataset.batch(64)

## 模型训练

In [20]:
EPOCHS = 5

# 训练 5 次
for epoch in range(EPOCHS):
    # 训练
    for (x_train, y_train) in train_dataset:
        train_step(model, optimizer, x_train, y_train)
        
    # 写入训练 log
    with train_summary_writer.as_default():
        tf.summary.scalar('loss', train_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

    # 测试
    for (x_test, y_test) in test_dataset:
        test_step(model, x_test, y_test)
        
    # 写入测试 log
    with test_summary_writer.as_default():
        tf.summary.scalar('loss', test_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)
    
    # 显示结果
    template = 'Epoch {}, Loss: {}, Accuracy: {}%, Test Loss: {}, Test Accuracy: {}%'
    print(template.format(epoch+1,
         train_loss.result(), 
         train_accuracy.result()*100,
         test_loss.result(), 
         test_accuracy.result()*100))

    # 重置效能衡量指标
    train_loss.reset_states()
    test_loss.reset_states()
    train_accuracy.reset_states()
    test_accuracy.reset_states()


Epoch 1, Loss: 0.28761035203933716, Accuracy: 91.67832946777344%, Test Loss: 0.13394859433174133, Test Accuracy: 96.01000213623047%
Epoch 2, Loss: 0.12773695588111877, Accuracy: 96.27999877929688%, Test Loss: 0.09488817304372787, Test Accuracy: 97.1500015258789%
Epoch 3, Loss: 0.0926021933555603, Accuracy: 97.22000122070312%, Test Loss: 0.07872872799634933, Test Accuracy: 97.52999877929688%
Epoch 4, Loss: 0.06980089098215103, Accuracy: 97.8516616821289%, Test Loss: 0.07566892355680466, Test Accuracy: 97.5%
Epoch 5, Loss: 0.05789392441511154, Accuracy: 98.20166778564453%, Test Loss: 0.0670073851943016, Test Accuracy: 97.89999389648438%


In [21]:
# 载入 TensorBoard notebook extension，即可在 jupyter notebook 启动 Tensorboard
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [22]:
# 启动 Tensorboard
%tensorboard --logdir logs/gradient_tape

Reusing TensorBoard on port 6006 (pid 12556), started 18:57:30 ago. (Use '!kill 12556' to kill it.)

## 使用浏览器输入以下网址，即可观看训练资讯：
## http://localhost:6006/

In [23]:
!taskkill /IM "tensorboard.exe" /F
# 或者使用以下指令，pid 以工作管理员查询
# !taskkill /F /PID pid

成功: 處理程序 "tensorboard.exe" (PID 20752) 已經終止了。


## 写入图片

In [25]:
# 任意找一张图片
img = x_train_norm[0].reshape((-1, 28, 28, 1))
img.shape

import datetime

# 指定 log 档名
logdir = ".\\logs\\train_data\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# Creates a file writer for the log directory.
file_writer = tf.summary.create_file_writer(logdir)

# Using the file writer, log the reshaped image.
with file_writer.as_default():
    # 将图片写入 log 档
    tf.summary.image("Training data", img, step=0)

In [26]:
%tensorboard --logdir logs/train_data

Reusing TensorBoard on port 6006 (pid 1744), started 0:17:10 ago. (Use '!kill 1744' to kill it.)